In [ ]:
import numpy as np
import random

def distance(city1, city2):
    """
    Calculate the Euclidean distance between two cities.
    city1, city2: Coordinates of the two cities.
    """
    return np.sqrt((city1[0] - city2[0])**2 + (city1[1] - city2[1])**2)

def total_distance(tour, cities):
    """
    Calculate the total distance of a tour.
    tour: Sequence of cities visited.
    cities: List of city coordinates.
    """
    dist = 0
    for i in range(len(tour)):
        dist += distance(cities[tour[i]], cities[tour[(i + 1) % len(tour)]])
    return dist

def initialize_pheromones(num_cities, initial_pheromone):
    """
    Initialize pheromone matrix.
    num_cities: Number of cities
    initial_pheromone: Initial pheromone level
    """
    return np.full((num_cities, num_cities), initial_pheromone)

def select_next_city(current_city, visited, pheromones, heuristic, alpha, beta):
    """
    Select the next city probabilistically based on pheromones and heuristic information.
    current_city: Current city of the ant
    visited: List of cities already visited
    pheromones: Pheromone matrix
    heuristic: Inverse distances matrix
    alpha: Importance of pheromones
    beta: Importance of heuristic information
    """
    probabilities = []
    for j in range(len(pheromones)):
        if j not in visited:
            tau = pheromones[current_city][j] ** alpha
            eta = heuristic[current_city][j] ** beta
            probabilities.append(tau * eta)
        else:
            probabilities.append(0)
    probabilities = np.array(probabilities) / sum(probabilities)
    return np.random.choice(range(len(pheromones)), p=probabilities)

def update_pheromones(pheromones, ants_tours, ants_distances, evaporation_rate):
    """
    Update pheromones based on ants' tours and evaporation rate.
    pheromones: Current pheromone matrix
    ants_tours: List of tours taken by ants
    ants_distances: List of distances for each ant's tour
    evaporation_rate: Rate at which pheromones evaporate
    """
    pheromones *= (1 - evaporation_rate)
    for i, tour in enumerate(ants_tours):
        contribution = 1 / ants_distances[i]
        for j in range(len(tour)):
            pheromones[tour[j]][tour[(j + 1) % len(tour)]] += contribution

def ant_colony_optimization(cities, num_ants, alpha, beta, evaporation_rate, initial_pheromone, num_iterations):
    """
    Ant Colony Optimization for the Traveling Salesman Problem.
    cities: List of city coordinates
    num_ants: Number of ants
    alpha: Importance of pheromones
    beta: Importance of heuristic information
    evaporation_rate: Evaporation rate of pheromones
    initial_pheromone: Initial pheromone value
    num_iterations: Number of iterations
    """
    num_cities = len(cities)
    pheromones = initialize_pheromones(num_cities, initial_pheromone)
    heuristic = np.array([[1 / distance(cities[i], cities[j]) if i != j else 0 for j in range(num_cities)] for i in range(num_cities)])
    best_tour = None
    best_distance = float('inf')

    for iteration in range(num_iterations):
        ants_tours = []
        ants_distances = []

        # Each ant constructs a solution
        for ant in range(num_ants):
            visited = [random.randint(0, num_cities - 1)]
            while len(visited) < num_cities:
                next_city = select_next_city(visited[-1], visited, pheromones, heuristic, alpha, beta)
                visited.append(next_city)
            ants_tours.append(visited)
            ants_distances.append(total_distance(visited, cities))

        # Update pheromones
        update_pheromones(pheromones, ants_tours, ants_distances, evaporation_rate)

        # Track the best solution
        current_best_index = np.argmin(ants_distances)
        if ants_distances[current_best_index] < best_distance:
            best_distance = ants_distances[current_best_index]
            best_tour = ants_tours[current_best_index]

        print(f"Iteration {iteration + 1}: Best Distance = {best_distance:.4f}")

    print("Optimization Complete!")
    return best_tour, best_distance

# Example Usage
CITIES = [
    (0, 0), (2, 6), (5, 5), (1, 3), (4, 8), (7, 2), (6, 6), (8, 3), (9, 7), (3, 1)
]
NUM_ANTS = 10
ALPHA = 1.0  # Importance of pheromones
BETA = 2.0   # Importance of heuristic information
EVAPORATION_RATE = 0.5
INITIAL_PHEROMONE = 0.1
NUM_ITERATIONS = 5

best_tour, best_distance = ant_colony_optimization(CITIES, NUM_ANTS, ALPHA, BETA, EVAPORATION_RATE, INITIAL_PHEROMONE, NUM_ITERATIONS)
print("Best Tour:", best_tour)
print("Best Distance:", best_distance)

Iteration 1: Best Distance = 36.0396
Iteration 2: Best Distance = 34.3199
Iteration 3: Best Distance = 33.5136
Iteration 4: Best Distance = 33.5136
Iteration 5: Best Distance = 29.7145
Optimization Complete!
Best Tour: [4, 2, 6, 8, 7, 5, 9, 0, 3, 1]
Best Distance: 29.714453801569597
